Sentiment analysis of movie (IMDB) reviews using dataset provided by the ACL 2011 paper, 
see http://ai.stanford.edu/~amaas/data/sentiment/
This notebook uses neural net models

The plan is to compare a variety of hyperparameters, vectorization techniques, neural net based models:
* dense neural network with bag of words
* dense neural network with fixed size input and words mapped to integers
* LSTM
* CNN


### Table of Contents<a class="anchor" id="table"></a>
* [Load data](#load)
* [Train different architectures](#train)
    * [Train NN 50 - 10 - 1](#train1)
    * [Train NN 256 - 128 - 1](#train2)
    * [Train NN with K-Fold cross validation](#kfold)
    * [Train RNN](#rnn)
* [Optimize](#opti)
    * [Optimize on dropouts](#opti_d)
        * no dropout
        * low dropout on 1 layer
        * high dropout on 1 layer
        * low dropout on 2 layers
        * high dropout on 2 layers
        * [Observation](#opti_d_o)

In [1]:
!pip install wget

In [2]:
import numpy as np
import os
import os.path
import glob
import time

import pandas as pd
import matplotlib as plt

from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
import nltk
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers
from tensorflow.keras import models
nltk.download('punkt')
import nltk
from sklearn.model_selection import train_test_split

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [0]:
from sklearn.model_selection import KFold

In [4]:
import wget
import tarfile

# By checking if the directory exists first, we allow people to delete the tarfile without the notebook re-downloading it
if os.path.isdir('aclImdb'):
    print("Dataset directory exists, taking no action")
else:    
    if not os.path.isfile('aclImdb_v1.tar.gz'):
        print("Downloading dataset")
        #!wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
        wget.download('http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz')
    else:
        print("Dataset already downloaded")
    
    print("Unpacking dataset")
    #!tar -xf aclImdb_v1.tar.gz 
    tar = tarfile.open("aclImdb_v1.tar.gz")
    tar.extractall()
    tar.close()
    print("Dataset unpacked in aclImdb")

Dataset directory exists, taking no action


In [0]:
def foundGPU():
  device_name = tf.test.gpu_device_name()
  if device_name != '/device:GPU:0':
    print('No GPU found')
    return False
  else: 
    print('Found GPU at: {}'.format(device_name))
    return True

In [6]:
# configuration
if foundGPU():
    SAMPLE_SIZE=5000
# Tried >7000 and the session crashes even before we start training the model which means 
# we might be doing something wrong with our preparation stage
#   SAMPLE_SIZE=12500    
else:
  SAMPLE_SIZE=2000

Found GPU at: /device:GPU:0


<a href='#table'>Back</a>
# Load data<a class="anchor" id="load"></a>

## Create a dense vector from reviews 

In [0]:
time_beginning_of_notebook = time.time()
SLICE = int(SAMPLE_SIZE / 2)
positive_file_list = glob.glob(os.path.join('aclImdb/train/pos', "*.txt"))
positive_sample_file_list = positive_file_list[:SLICE]

negative_file_list = glob.glob(os.path.join('aclImdb/train/neg', "*.txt"))
negative_sample_file_list = negative_file_list[:SLICE]

import re

# load doc into memory
# regex to clean markup elements 
def load_doc(filename):
    # open the file as read only
    file = open(filename, 'r', encoding='utf8')
    # read all text
    text = re.sub('<[^>]*>', ' ', file.read())
    #text = file.read()
    # close the file
    file.close()
    return text

In [0]:
positive_strings = [load_doc(x) for x in positive_sample_file_list]
negative_strings = [load_doc(x) for x in negative_sample_file_list]

positive_tokenized = [word_tokenize(s) for s in positive_strings]
negative_tokenized = [word_tokenize(s) for s in negative_strings]

In [0]:
from collections import Counter
import numpy as np

In [0]:
total_counts = Counter()
all_reviews = positive_tokenized + negative_tokenized
for r in all_reviews:
    for word in r:
        total_counts[word] += 1

In [0]:
vocab = set(total_counts.keys())

In [12]:
vocab_size = len(vocab)
print(vocab_size)

53956


In [0]:
# Create a dictionary of words in the vocabulary mapped to index positions
# (to be used in layer_0)
word2index = {}
for i,word in enumerate(vocab):
    word2index[word] = i

In [14]:
print("ID of 'movie' = {}".format(word2index['movie']))

ID of 'movie' = 36827


In [0]:
def convert_to_bag(review):
    bag = np.zeros(vocab_size)
    for word in review:
        i = word2index[word]
        bag[i]+=1
    return bag

In [0]:
test_bag = convert_to_bag(all_reviews[0])

In [0]:
all_reviews_encoded = [convert_to_bag(x) for x in all_reviews]

In [18]:
all_reviews_encoded[0].shape

(53956,)

In [0]:
import random

positive_labels = []
for i in range(len(positive_tokenized)):
    positive_labels.append('POSITIVE')
negative_labels = []
for i in range(len(negative_tokenized)):
    negative_labels.append('NEGATIVE')
   

In [0]:
labels = positive_labels + negative_labels

num_lables = []

for val in labels:
    if val == 'POSITIVE':
       num_lables.append(1)
    else:
       num_lables.append(0) 
    

In [0]:
reviews_and_labels = list(zip(all_reviews_encoded, num_lables))
random.shuffle(reviews_and_labels)
reviews, labels = zip(*reviews_and_labels)

In [0]:
labels = np.array(labels)

In [0]:
def saveTrainingMetrics(title):
  df = pd.DataFrame(results.history)
  df=df[df['val_acc']==df.val_acc.max()]
  df.reset_index(inplace=True)
  df["title"]=[title]
  df["sample_size"]=[SAMPLE_SIZE]
  df["vocab_size"]=[vocab_size]  
  df["nb_epochs"]=[df.iloc[0]["index"]+1]
  df.drop(labels="index",axis=1,inplace=True)
  print(df)
  df.to_csv(path_or_buf=df.iloc[0].title+".csv")

## Create a sparse matrix from reviews (where we keep the order of the words)

In [24]:
positive_strings[0]

'First, nobody can understand why this movie is rated so poorly. Not only is this the first real horrific movie since a very long time for me who am pretty hard-boiled with a decades long experience of horror starting with driving through dark rides (ghost trains) as a child. Second, the main actress Cheri Christian has a face that lets you hope she will be the leading actress in major pictures of the future. Third, this woman is that tremendously beautiful that I suggest the directors retire all those Cameron Diazes, Eva Mendezes, and how ever the names of these ephemeral bulb-lights are. Mrs. Christian is not a light, but a sun.  However, "Dark remains" is also of considerable metaphysical importance. They idea that photographs shows creatures of the intermediary reign between reality and "imagination" that are not visible with one\' own eyes is not new. But I have never seen in a movie before that those creatures are visible on the photographs only for certain people and only to cer

In [0]:
### We need to found out why we need this code and possibly who has written this code
# reviews=[]
# for sentence in positive_strings:
#     reviews.append([sentence,1])
# for sentence in negative_strings:
#     reviews.append([sentence,0])
# random.shuffle(reviews)


In [0]:
# tokenizer = keras.preprocessing.text.Tokenizer(num_words=None, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~',\
#                                    lower=True, split=' ', char_level=False, oov_token=None, document_count=0)

In [0]:
# len(reviews)

<a href='#table'>Back</a>
# Train models<a class="anchor" id="train"></a>
## Train NN 50 - 10 - 1 <a class="anchor" id="train1"></a>


In [0]:
X_train, X_test, y_train, y_test = train_test_split(np.array(reviews), np.array(labels), test_size=0.25)

In [0]:
X_train

In [0]:
model = keras.Sequential()
model.add(layers.Dense(50, activation = "relu", input_shape=(vocab_size, )))
model.add(layers.Dense(10, activation = "relu"))
model.add(layers.Dense(1, activation = "sigmoid"))
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 50)                3193400   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                510       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 11        
Total params: 3,193,921
Trainable params: 3,193,921
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(
 optimizer = "adam",
 loss = "binary_crossentropy",
 metrics = ["accuracy"]
)

In [0]:
results = model.fit(
 X_train, y_train,
 epochs=20,
 validation_data=(X_test, y_test),
 batch_size=500
)

Train on 5250 samples, validate on 1750 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/20
5250/5250 [==============================] - 3s 631us/sample - loss: 0.6058 - acc: 0.7080 - val_loss: 0.4936 - val_acc: 0.8309
Epoch 2/20
5250/5250 [==============================] - 3s 480us/sample - loss: 0.3472 - acc: 0.9183 - val_loss: 0.3708 - val_acc: 0.8697
Epoch 3/20
5250/5250 [==============================] - 3s 482us/sample - loss: 0.1877 - acc: 0.9632 - val_loss: 0.3177 - val_acc: 0.8754
Epoch 4/20
5250/5250 [==============================] - 3s 484us/sample - loss: 0.1008 - acc: 0.9872 - val_loss: 0.3001 - val_acc: 0.8857
Epoch 5/20
5250/5250 [==============================] - 3s 482us/sample - loss: 0.0576 - acc: 0.9943 - val_loss: 0.3010 - val_acc: 0.8846
Epoch 6/20
5250/5250 [==============================] - 3s 480us/sample - loss: 0.0356 - acc: 0.9970 - val_loss: 0.3031 - val_acc: 0.8840
Epoch 7/20
5250/5250 [==============================] - 3s 481us/sample - lo

In [0]:
saveTrainingMetrics("Opti-NN-Train NN 50 - 10 - 1")

        acc     loss   val_acc  val_loss                         title  \
0  0.987238  0.10084  0.885714  0.300149  Opti-NN-Train NN 50 - 10 - 1   

   sample_size  vocab_size  nb_epochs  
0         7000       63867          4  


## Train NN 256 - 128 - 1 <a class="anchor" id="train2"></a>

In [0]:
model = keras.Sequential([
    layers.Dense(256, activation = "relu", input_shape=(vocab_size, )),
    layers.Dense(128, activation = "relu"),
    layers.Dense(1, activation = "sigmoid")
])

In [0]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 256)               16350208  
_________________________________________________________________
dense_4 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 129       
Total params: 16,383,233
Trainable params: 16,383,233
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(
 optimizer = "adam",
 loss = "binary_crossentropy",
 metrics = ["accuracy"]
)

In [0]:
results = model.fit(
 X_train, y_train,
 epochs= 1,
 validation_data=(X_test, y_test),
batch_size=500
)

Train on 5250 samples, validate on 1750 samples
5250/5250 [==============================] - 3s 597us/sample - loss: 0.6035 - acc: 0.6979 - val_loss: 0.4399 - val_acc: 0.8343


In [0]:
results.history

{'acc': [0.69790477],
 'loss': [0.6035325229167938],
 'val_acc': [0.83428574],
 'val_loss': [0.43990913884980337]}

In [0]:
saveTrainingMetrics("Opti-NN-Train NN 256 - 128 - 1")

        acc      loss   val_acc  val_loss                           title  \
0  0.697905  0.603533  0.834286  0.439909  Opti-NN-Train NN 256 - 128 - 1   

   sample_size  vocab_size  nb_epochs  
0         7000       63867          1  


## Train NN with K-Fold cross validation <a class="anchor" id="kfold"></a>

In [0]:
X_train, X_test, y_train, y_test = train_test_split(np.array(reviews), np.array(labels), test_size=0.25)

In [0]:
kfold = KFold(3, True, 1)

In [0]:
train_data = list(zip(X_train, y_train))

In [0]:
train_data[0][0].shape

In [0]:
histories=[]
for train_indices, test_indices in kfold.split(X_train,y=y_train):
    model = keras.Sequential([
    layers.Dense(256, activation = "relu", input_shape=(vocab_size, )),
    layers.Dense(128, activation = "relu"),
    layers.Dense(1, activation = "sigmoid")
    ])
    model.compile(
     optimizer = "adam",
     loss = "binary_crossentropy",
     metrics = ["accuracy"]
    )
    K_X_train = X_train[train_indices]
    K_y_train = y_train[train_indices]
    K_X_test = X_train[test_indices]
    K_y_test = y_train[test_indices]
    results=model.fit(
        K_X_train, K_y_train,
        epochs= 5,
        validation_data=(K_X_test, K_y_test),
        batch_size=128
    )
    histories.append(results.history)

In [30]:
saveTrainingMetrics("Opti-NN-Train NN with K-Fold cross validation")

      acc      loss  val_acc  val_loss  \
0  0.9828  0.135673   0.8648   0.34739   

                                           title  sample_size  vocab_size  \
0  Opti-NN-Train NN with K-Fold cross validation         5000       53956   

   nb_epochs  
0          5  


In [0]:

df = pd.DataFrame(data=histories)
for col in df.columns:
    df[col] =  df[col].apply(lambda x: x[-1])
plot=df[["acc","val_acc"]].plot()
plot.set_ylim([0,1])

means=df[["acc","val_acc"]].mean()
print("mean acc: {}, mean val_acc: {}".format(means["acc"],means["val_acc"]))


## Train RNN <a class="anchor" id="rnn"></a>

<a href='#table'>Back</a>
# OPTIMIZE<a class="anchor" id="opti"></a>

## Optimize on dropout<a class="anchor" id="opti_d"></a>

In [0]:
X_train, X_test, y_train, y_test = train_test_split(np.array(reviews), np.array(labels), test_size=0.25)

In [0]:
kfold = KFold(3, True, 1)

In [0]:
train_data = list(zip(X_train, y_train))

In [0]:
from pdb import set_trace

def getMeansFromResultsHistory(histories):
  df = pd.DataFrame(data=histories)
  for col in df.columns:
      df[col] =  df[col].apply(lambda x: x[-1])
  means=df[["acc","val_acc"]].mean()
  return means

def trainModelWithDropoutOn1Layer(epochs_nb=5,rate=0.0):
  histories=[]
  for train_indices, test_indices in kfold.split(X_train,y=y_train):
      model = keras.Sequential([
      layers.Dense(256, activation = "relu", input_shape=(vocab_size, )),
      layers.Dropout(rate),
      layers.Dense(128, activation = "relu"),
      layers.Dense(1, activation = "sigmoid")
      ])
      model.compile(
       optimizer = "adam",
       loss = "binary_crossentropy",
       metrics = ["accuracy"]
      )
      K_X_train = X_train[train_indices]
      K_y_train = y_train[train_indices]
      K_X_test = X_train[test_indices]
      K_y_test = y_train[test_indices]
      results=model.fit(
          K_X_train, K_y_train,
          epochs= epochs_nb,
          validation_data=(K_X_test, K_y_test),
          batch_size=1000
      )
      histories.append(results.history)
#       set_trace()
  
  means=getMeansFromResultsHistory(histories)
  print(means) 
  return results,means


def trainModelWithDropoutOn2Layers(epochs_nb=5,rate=0.0):
  histories=[]
  for train_indices, test_indices in kfold.split(X_train,y=y_train):
      model = keras.Sequential([
      layers.Dense(256, activation = "relu", input_shape=(vocab_size, )),
      layers.Dropout(rate),
      layers.Dense(128, activation = "relu"),
      layers.Dropout(rate),
      layers.Dense(1, activation = "sigmoid")
      ])
      model.compile(
       optimizer = "adam",
       loss = "binary_crossentropy",
       metrics = ["accuracy"]
      )
      K_X_train = X_train[train_indices]
      K_y_train = y_train[train_indices]
      K_X_test = X_train[test_indices]
      K_y_test = y_train[test_indices]
      results=model.fit(
          K_X_train, K_y_train,
          epochs= epochs_nb,
          validation_data=(K_X_test, K_y_test),
          batch_size=1000
      )
      histories.append(results.history)
#       set_trace()
  
  means=getMeansFromResultsHistory(histories)
  print(means) 
  return results,means

In [0]:
dropout_means=[]

### No dropout


In [39]:
rate=0.0
results,means=trainModelWithDropoutOn1Layer(epochs_nb=5,rate=rate)
dropout_means.append([means["acc"],means["val_acc"], rate,0])

Train on 2500 samples, validate on 1250 samples
Epoch 1/5
2500/2500 [==============================] - 2s 639us/sample - loss: 0.6808 - acc: 0.5704 - val_loss: 0.6432 - val_acc: 0.6960
Epoch 2/5
2500/2500 [==============================] - 1s 476us/sample - loss: 0.5131 - acc: 0.8496 - val_loss: 0.5145 - val_acc: 0.7696
Epoch 3/5
2500/2500 [==============================] - 1s 483us/sample - loss: 0.3129 - acc: 0.9352 - val_loss: 0.4295 - val_acc: 0.8184
Epoch 4/5
2500/2500 [==============================] - 1s 478us/sample - loss: 0.1704 - acc: 0.9716 - val_loss: 0.3919 - val_acc: 0.8400
Epoch 5/5
2500/2500 [==============================] - 1s 474us/sample - loss: 0.0893 - acc: 0.9888 - val_loss: 0.3646 - val_acc: 0.8552
Train on 2500 samples, validate on 1250 samples
Epoch 1/5
2500/2500 [==============================] - 2s 642us/sample - loss: 0.7129 - acc: 0.5056 - val_loss: 0.6507 - val_acc: 0.5568
Epoch 2/5
2500/2500 [==============================] - 1s 476us/sample - loss: 0.5

In [40]:
saveTrainingMetrics("Opti-NN-Optimise - No dropout")

      acc      loss  val_acc  val_loss                          title  \
0  0.9884  0.091937   0.8624  0.333578  Opti-NN-Optimise - No dropout   

   sample_size  vocab_size  nb_epochs  
0         5000       53956          5  


### Low dropout on 1 layer

In [41]:
rate=0.2
results,means=trainModelWithDropoutOn1Layer(epochs_nb=5,rate=rate)
dropout_means.append([means["acc"],means["val_acc"], rate,1])

Train on 2500 samples, validate on 1250 samples
Epoch 1/5
2500/2500 [==============================] - 2s 663us/sample - loss: 0.6885 - acc: 0.5356 - val_loss: 0.6566 - val_acc: 0.6448
Epoch 2/5
2500/2500 [==============================] - 1s 491us/sample - loss: 0.5436 - acc: 0.8168 - val_loss: 0.5365 - val_acc: 0.7528
Epoch 3/5
2500/2500 [==============================] - 1s 478us/sample - loss: 0.3543 - acc: 0.9180 - val_loss: 0.4420 - val_acc: 0.8144
Epoch 4/5
2500/2500 [==============================] - 1s 478us/sample - loss: 0.2070 - acc: 0.9652 - val_loss: 0.3749 - val_acc: 0.8440
Epoch 5/5
2500/2500 [==============================] - 1s 477us/sample - loss: 0.1151 - acc: 0.9824 - val_loss: 0.3543 - val_acc: 0.8568
Train on 2500 samples, validate on 1250 samples
Epoch 1/5
2500/2500 [==============================] - 2s 672us/sample - loss: 0.7339 - acc: 0.5176 - val_loss: 0.6854 - val_acc: 0.5160
Epoch 2/5
2500/2500 [==============================] - 1s 479us/sample - loss: 0.6

In [42]:
saveTrainingMetrics("Opti-NN-Optimise - Low dropout on 1 layer")

      acc      loss  val_acc  val_loss  \
0  0.9844  0.108719   0.8656  0.328614   

                                       title  sample_size  vocab_size  \
0  Opti-NN-Optimise - Low dropout on 1 layer         5000       53956   

   nb_epochs  
0          5  


### High dropout on 1 layer


In [43]:
rate=0.4
results,means=trainModelWithDropoutOn1Layer(epochs_nb=5,rate=rate)
dropout_means.append([means["acc"],means["val_acc"], rate,1])

Train on 2500 samples, validate on 1250 samples
Epoch 1/5
2500/2500 [==============================] - 2s 688us/sample - loss: 0.7076 - acc: 0.5292 - val_loss: 0.6660 - val_acc: 0.6048
Epoch 2/5
2500/2500 [==============================] - 1s 483us/sample - loss: 0.6176 - acc: 0.7212 - val_loss: 0.5910 - val_acc: 0.7624
Epoch 3/5
2500/2500 [==============================] - 1s 479us/sample - loss: 0.4786 - acc: 0.8880 - val_loss: 0.4995 - val_acc: 0.7888
Epoch 4/5
2500/2500 [==============================] - 1s 479us/sample - loss: 0.3323 - acc: 0.9208 - val_loss: 0.4216 - val_acc: 0.8264
Epoch 5/5
2500/2500 [==============================] - 1s 479us/sample - loss: 0.2163 - acc: 0.9536 - val_loss: 0.3728 - val_acc: 0.8440
Train on 2500 samples, validate on 1250 samples
Epoch 1/5
2500/2500 [==============================] - 2s 698us/sample - loss: 0.7165 - acc: 0.5212 - val_loss: 0.6603 - val_acc: 0.5624
Epoch 2/5
2500/2500 [==============================] - 1s 474us/sample - loss: 0.6

In [44]:
saveTrainingMetrics("Opti-NN-Optimise - High dropout on 1 layer")

      acc      loss  val_acc  val_loss  \
0  0.9488  0.272705   0.8512  0.382278   

                                        title  sample_size  vocab_size  \
0  Opti-NN-Optimise - High dropout on 1 layer         5000       53956   

   nb_epochs  
0          5  


### Low dropout on 2 layers

### High dropout on 2 layers


Plot results

In [0]:
df = pd.DataFrame(data=dropout_means,columns=['acc','val_acc','rate','nb_layers'])
plt.rcParams["figure.figsize"] = [17,2]
plot=df[["acc","val_acc"]].plot()
plot.set_ylim([0.7,1])
plot.grid()

plt.rcParams["figure.figsize"] = [17,1.5]
plot=df[["rate"]].plot()
plot.set_ylim([0,0.5])
plot.grid()

plt.rcParams["figure.figsize"] = [17,1.5]
plot=df[["nb_layers"]].plot()
plot.set_ylim([0,2.2])
plot.grid()

In [0]:
dropout_means

### Observation<a class="anchor" id="opti_d_o"></a>
we have similar results, but got a higher test accuracy with low dropout on all layers and also less overfit (training and test accuracies are closer)